# ssp.train.py

In [4]:
import torch   
import torch.nn as nn 
import torch.nn.functional as F   
import numpy as np
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from scipy import io
import itertools
import math
import datetime
import wandb
import pickle
import json
import time
import sys
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from scipy.optimize import linear_sum_assignment
from snntorch import spikegen
from sklearn.manifold import TSNE


# my module import
from modules import *

# os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

# modules 폴더에 새모듈.py 만들면
# modules/__init__py 파일에 form .새모듈 import * 하셈
# 그리고 새모듈.py에서 from modules.새모듈 import * 하셈

In [5]:
def cluster_train_system( 
    gpu = '4',
    Conv_net = True,
    SAE_net = True,

    # hyperparameter
    dataset_num = 16,
    spike_length = 50,
    num_cluster = 4,  # 클러스터 수 설정 # 논문엔 4개라는데 여기서는 3개로 했네
    training_cycle = 2400, # 그 초기 몇개까지만 cluster update할지


    batch_size = 32,
    max_epoch = 7000,
    learning_rate = 0.001,
    normalize_on = False, # True or False #이거 안 씀 # 이거 별로 안 좋은 normalize같음 # 쓸 거면 다른 거 써라.
    need_bias = False,
    # first_layer_no_train = False
    lif_add_at_first = False,
    my_seed = 42,

    TIME = 10, # SAE일 때만 유효
    v_decay = 0.5,
    v_threshold = 0.5,
    v_reset = 10000.0, # 10000이상 일 시 hard reset
    BPTT_on = True,

    SAE_hidden_nomean = True,
    current_time = '20250101_210938_786',

    optimizer = 'Adam',
    coarse_com_mode = True,
    coarse_com_config = (2.0, -2.0), # (max, min)

    sae_l2_norm_bridge = True,
    sae_lif_bridge = False,

    accuracy_check_epoch_term = 5,
    
    lif_add_at_last = False,

    two_channel_input = False,

    lateral_feature_num = 4,

    lc_adc_on = False, 

    converted_net_forward = False,

    pretrained_net = None, 

    vth_mul_on = False,
    batch_norm_on = False,

    l2_norm_loss_weight = 0.0,

    QCFS_neuron_on = False,

    quantize_level_num = 0,

    fusion_net = False, # True False
    repeat_coding = False,
    
    sae_relu_on = False,

    conv1d_scaling = False,

    norm01 = True,

    chan_loss_factor = 1.0,
    ):
    if coarse_com_mode == True:
        assert coarse_com_config[0] > coarse_com_config[1], 'coarse_com_config[0] > coarse_com_config[1]이어야 함'
        assert converted_net_forward == False
        # assert SAE_net == True, 'coarse_com_mode는 SAE_net이 True일 때만 가능'
    if two_channel_input == True:
        assert Conv_net and coarse_com_mode, 'two_channel_input는 Conv_net이 True일 때만 가능'
    if lc_adc_on == True:
        assert coarse_com_mode and SAE_net, 'lc_adc_on은 coarse_com_mode와 SAE_net이 True일 때만 가능'
    if converted_net_forward == True:
        assert SAE_net == False, 'converted_net_forward는 SAE_net이 False일 때만 가능'
    if conv1d_scaling:
        assert Conv_net and coarse_com_mode and normalize_on
    seed_assign(my_seed)
    ## 함수 내 모든 로컬 변수 저장 ########################################################
    hyperparameters = locals()
    print(hyperparameters)
    # JSON으로 저장
    with open(f"result_save/cluster_accuracy_history_{current_time}.json", 'w') as f:
        json.dump(hyperparameters, f, indent=4)
    ######################################################################################

    
    wandb.config.update(hyperparameters)
    wandb.run.name = f'{current_time}_SAE_net_{SAE_net}_v_threshold_{v_threshold}'

    data_path = ["/data2/spike_sorting/neuropixels_choi/set1/20141202_all_es_merged_3125000_limit_autoencoder_data.pt",]
    label_path = ["/data2/spike_sorting/neuropixels_choi/set1/20141202_all_es_merged_3125000_limit_autoencoder_label.pt"]

    AE_train_data = data_path[0] #AE_train_path_gt_detect #AE_train_path_real_detect

    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
    os.environ["CUDA_VISIBLE_DEVICES"]= gpu


    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


    if coarse_com_mode == True:
        level_num = TIME
        TIME = spike_length
        spike_length = level_num
        level_interval = (coarse_com_config[0] - coarse_com_config[1]) / (level_num-1)  # max - min
        levels = [coarse_com_config[1] + level_interval * i for i in range(level_num)]
        levels = torch.tensor(levels).to(torch.float32).to(device)
        levels = levels.repeat(TIME,1) 
        # print('levels', levels, levels.shape) # TIME, level_num

    n_sample = spike_length

    class spikedataset(Dataset):
        def __init__(self, path, transform = None):    
            self.transform = transform
            self.spike = torch.load(path)
            
        def __getitem__(self, index):
            spike = self.spike[index]            
            if self.transform is not None:
                spike = self.transform(spike)
            return spike
        
        def __len__(self):
            return len(self.spike)

    train_dataset = spikedataset(AE_train_data)
    train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

    # 모델 초기화
    if SAE_net == False: # 여기서는 l2norm, lif bridge 둘 다 true면 l2norm먼저
        assert two_channel_input == False

        if Conv_net == True:
            # input_channels = 2 if two_channel_input else 1
            input_channels = TIME if coarse_com_mode else 1
            if fusion_net == True:
                assert False, '이거 맞음? 다시 확인'
                net = FUSION_net_conv1(input_channels=input_channels, input_length=n_sample, encoder_ch = [32, 64, 96], fc_dim = lateral_feature_num, padding = 0, stride = 2, kernel_size = 3, 
                                    synapse_fc_trace_const1=1, 
                                    synapse_fc_trace_const2=v_decay, #안씀 
                                    TIME=TIME, v_init=0.0, v_decay=v_decay, v_threshold=v_threshold, v_reset=v_reset, 
                                    sg_width=4.0, surrogate='sigmoid', BPTT_on=BPTT_on, need_bias=need_bias, lif_add_at_first=lif_add_at_first,
                                    sae_l2_norm_bridge = sae_l2_norm_bridge, sae_lif_bridge = sae_lif_bridge, lif_add_at_last=lif_add_at_last, repeat_coding=repeat_coding).to(device)
            else: 
                net = Autoencoder_conv1(input_channels=input_channels, input_length=n_sample, encoder_ch = [32, 64, 96], fc_dim = lateral_feature_num, padding = 0, stride = 2, kernel_size = 3, need_bias=need_bias, l2norm_bridge=sae_l2_norm_bridge, relu_bridge=sae_lif_bridge, activation_collector_on=False,
                                        batch_norm_on=batch_norm_on, QCFS_neuron_on=QCFS_neuron_on).to(device)
            net = torch.nn.DataParallel(net)
            if converted_net_forward:
                converted_net = SAE_converted_conv1(input_channels=input_channels, input_length=n_sample, encoder_ch = [32, 64, 96], fc_dim = lateral_feature_num, padding = 0, stride = 2, kernel_size = 3, 
                                    synapse_fc_trace_const1=1, 
                                    synapse_fc_trace_const2=v_decay, #안씀 
                                    TIME=TIME, v_init=0.0, v_decay=v_decay, v_threshold=v_threshold, v_reset=v_reset, 
                                    sg_width=4.0, surrogate='sigmoid', BPTT_on=BPTT_on, need_bias=need_bias, lif_add_at_first=lif_add_at_first,
                                    sae_l2_norm_bridge = sae_l2_norm_bridge, sae_lif_bridge = sae_lif_bridge, lif_add_at_last=lif_add_at_last,
                                    vth_mul_on=vth_mul_on, batch_norm_on=batch_norm_on).to(device) # lif bridge는 무조건 들어가게 해놨음.
                converted_net = torch.nn.DataParallel(converted_net)
                print('converted_net', converted_net)
        else:
            n_sample = n_sample * TIME if coarse_com_mode else n_sample
            net = Autoencoder_only_FC(encoder_ch=[400, lateral_feature_num], decoder_ch=[400,n_sample], n_sample=n_sample, need_bias=need_bias, l2norm_bridge=sae_l2_norm_bridge, relu_bridge=sae_lif_bridge, activation_collector_on=False,
                                    batch_norm_on=batch_norm_on, QCFS_neuron_on=QCFS_neuron_on).to(device)
            net = torch.nn.DataParallel(net)
            if converted_net_forward:
                converted_net = SAE_converted_fc(encoder_ch=[400, lateral_feature_num], 
                                    decoder_ch=[400, n_sample], 
                                    in_channels=n_sample, # in_channel 이 여기선 걍 lenght.
                                    synapse_fc_trace_const1=1,
                                    synapse_fc_trace_const2=v_decay,  #안씀 
                                    TIME=TIME, v_init=0.0, v_decay=v_decay, v_threshold=v_threshold, v_reset=v_reset, 
                                    sg_width=4.0, surrogate='sigmoid', BPTT_on=BPTT_on, need_bias=need_bias, lif_add_at_first=lif_add_at_first,
                                    sae_l2_norm_bridge = sae_l2_norm_bridge, sae_lif_bridge = sae_lif_bridge, lif_add_at_last=lif_add_at_last,
                                    vth_mul_on=vth_mul_on, batch_norm_on=batch_norm_on).to(device) # lif bridge는 무조건 들어가게 해놨음.
                converted_net = torch.nn.DataParallel(converted_net)
                # print('converted_net', converted_net)
    else:
        if Conv_net == True: 
            input_channels = 1
            input_channels = 2 if two_channel_input else 1
            if fusion_net == True:  
                assert coarse_com_mode == True
                # net = SAE_FUSION2_net_conv1(input_channels=input_channels, input_length=n_sample, encoder_ch = [32, 64, 96], fc_dim = lateral_feature_num, padding = 0, stride = 2, kernel_size = 3, 
                #                     synapse_fc_trace_const1=1, 
                #                     synapse_fc_trace_const2=v_decay, #안씀 
                #                     TIME=TIME, v_init=0.0, v_decay=v_decay, v_threshold=v_threshold, v_reset=v_reset, 
                #                     sg_width=4.0, surrogate='sigmoid', BPTT_on=BPTT_on, need_bias=need_bias, lif_add_at_first=lif_add_at_first,
                #                     sae_l2_norm_bridge = sae_l2_norm_bridge, sae_lif_bridge = sae_lif_bridge, lif_add_at_last=lif_add_at_last, batch_norm_on=batch_norm_on, sae_relu_on=sae_relu_on).to(device)
                # net = SAE_FUSION3_net_conv1(input_channels=input_channels, input_length=n_sample, encoder_ch = [32, 64, 96], fc_dim = lateral_feature_num, padding = 0, stride = 2, kernel_size = 3, 
                #                     synapse_fc_trace_const1=1, 
                #                     synapse_fc_trace_const2=v_decay, #안씀 
                #                     TIME=TIME, v_init=0.0, v_decay=v_decay, v_threshold=v_threshold, v_reset=v_reset, 
                #                     sg_width=4.0, surrogate='sigmoid', BPTT_on=BPTT_on, need_bias=need_bias, lif_add_at_first=lif_add_at_first,
                #                     sae_l2_norm_bridge = sae_l2_norm_bridge, sae_lif_bridge = sae_lif_bridge, lif_add_at_last=lif_add_at_last, batch_norm_on=batch_norm_on, sae_relu_on=sae_relu_on).to(device)
                # net = SAE_FUSION4_net_conv1(input_channels=input_channels, input_length=n_sample, encoder_ch = [32, 64, 96], fc_dim = lateral_feature_num, padding = 0, stride = 2, kernel_size = 3, 
                #                     synapse_fc_trace_const1=1, 
                #                     synapse_fc_trace_const2=v_decay, #안씀 
                #                     TIME=TIME, v_init=0.0, v_decay=v_decay, v_threshold=v_threshold, v_reset=v_reset, 
                #                     sg_width=4.0, surrogate='sigmoid', BPTT_on=BPTT_on, need_bias=need_bias, lif_add_at_first=lif_add_at_first,
                #                     sae_l2_norm_bridge = sae_l2_norm_bridge, sae_lif_bridge = sae_lif_bridge, lif_add_at_last=lif_add_at_last, batch_norm_on=batch_norm_on, sae_relu_on=sae_relu_on).to(device)
                net = SAE_FUSION5_net_conv1(input_channels=input_channels, input_length=n_sample, encoder_ch = [32, 64, 96], fc_dim = lateral_feature_num, padding = 0, stride = 2, kernel_size = 3, 
                                    synapse_fc_trace_const1=1, 
                                    synapse_fc_trace_const2=v_decay, #안씀 
                                    TIME=TIME, v_init=0.0, v_decay=v_decay, v_threshold=v_threshold, v_reset=v_reset, 
                                    sg_width=4.0, surrogate='sigmoid', BPTT_on=BPTT_on, need_bias=need_bias, lif_add_at_first=lif_add_at_first,
                                    sae_l2_norm_bridge = sae_l2_norm_bridge, sae_lif_bridge = sae_lif_bridge, lif_add_at_last=lif_add_at_last, batch_norm_on=batch_norm_on, sae_relu_on=sae_relu_on).to(device)
                # net = SAE_FUSION6_net_conv1(input_channels=input_channels, input_length=n_sample, encoder_ch = [32, 64, 96], fc_dim = lateral_feature_num, padding = 0, stride = 2, kernel_size = 3, 
                #                     synapse_fc_trace_const1=1, 
                #                     synapse_fc_trace_const2=v_decay, #안씀 
                #                     TIME=TIME, v_init=0.0, v_decay=v_decay, v_threshold=v_threshold, v_reset=v_reset, 
                #                     sg_width=4.0, surrogate='sigmoid', BPTT_on=BPTT_on, need_bias=need_bias, lif_add_at_first=lif_add_at_first,
                #                     sae_l2_norm_bridge = sae_l2_norm_bridge, sae_lif_bridge = sae_lif_bridge, lif_add_at_last=lif_add_at_last, batch_norm_on=batch_norm_on, sae_relu_on=sae_relu_on).to(device)
                # net = SAE_FUSION7_net_conv1(input_channels=input_channels, input_length=n_sample, encoder_ch = [32, 64, 96], fc_dim = lateral_feature_num, padding = 0, stride = 2, kernel_size = 3, 
                #                     synapse_fc_trace_const1=1, 
                #                     synapse_fc_trace_const2=v_decay, #안씀 
                #                     TIME=TIME, v_init=0.0, v_decay=v_decay, v_threshold=v_threshold, v_reset=v_reset, 
                #                     sg_width=4.0, surrogate='sigmoid', BPTT_on=BPTT_on, need_bias=need_bias, lif_add_at_first=lif_add_at_first,
                #                     sae_l2_norm_bridge = sae_l2_norm_bridge, sae_lif_bridge = sae_lif_bridge, lif_add_at_last=lif_add_at_last, batch_norm_on=batch_norm_on, sae_relu_on=sae_relu_on).to(device)
            else:
                net = SAE_conv1(input_channels=input_channels, input_length=n_sample, encoder_ch = [32, 64, 96], fc_dim = lateral_feature_num, padding = 0, stride = 2, kernel_size = 3, 
                                    synapse_fc_trace_const1=1, 
                                    synapse_fc_trace_const2=v_decay, #안씀 
                                    TIME=TIME, v_init=0.0, v_decay=v_decay, v_threshold=v_threshold, v_reset=v_reset, 
                                    sg_width=4.0, surrogate='sigmoid', BPTT_on=BPTT_on, need_bias=need_bias, lif_add_at_first=lif_add_at_first,
                                    sae_l2_norm_bridge = sae_l2_norm_bridge, sae_lif_bridge = sae_lif_bridge, lif_add_at_last=lif_add_at_last, batch_norm_on=batch_norm_on, sae_relu_on=sae_relu_on).to(device)
            # net = SAE_conv1_DR(input_channels=input_channels, input_length=n_sample, encoder_ch = [32, 64, 96], fc_dim = lateral_feature_num, padding = 0, stride = 2, kernel_size = 3, 
            #                     synapse_fc_trace_const1=1, 
            #                     synapse_fc_trace_const2=v_decay, #안씀 
            #                     TIME=TIME, v_init=0.0, v_decay=v_decay, v_threshold=v_threshold, v_reset=v_reset, 
            #                     sg_width=4.0, surrogate='sigmoid', BPTT_on=BPTT_on, need_bias=need_bias, lif_add_at_first=lif_add_at_first,
            #                     sae_l2_norm_bridge = sae_l2_norm_bridge, sae_lif_bridge = sae_lif_bridge, lif_add_at_last=lif_add_at_last, batch_norm_on=batch_norm_on).to(device)
            net = torch.nn.DataParallel(net)
        else:
            net = SAE_fc_only(encoder_ch=[400, lateral_feature_num], 
                                decoder_ch=[400, n_sample], 
                                in_channels=n_sample, # in_channel 이 여기선 걍 lenght.
                                synapse_fc_trace_const1=1,
                                synapse_fc_trace_const2=v_decay,  #안씀 
                                TIME=TIME, v_init=0.0, v_decay=v_decay, v_threshold=v_threshold, v_reset=v_reset, 
                                sg_width=4.0, surrogate='sigmoid', BPTT_on=BPTT_on, need_bias=need_bias, lif_add_at_first=lif_add_at_first,
                                sae_l2_norm_bridge = sae_l2_norm_bridge, sae_lif_bridge = sae_lif_bridge, lif_add_at_last=lif_add_at_last, batch_norm_on=batch_norm_on, sae_relu_on=sae_relu_on).to(device)
            net = torch.nn.DataParallel(net)

    # net = torch.load('/home/bhkim003/github_folder/ByeonghyeonKim/my_snn/net_save/save_now_net_AE_re_e7000.pth')
    # net = torch.load('/home/bhkim003/github_folder/ByeonghyeonKim/my_snn/net_save/save_now_net_20250101_210938_786.pth')
    # load했으면 torch.nn.DataParallel 하지마
    # net.module.load_state_dict(torch.load('/home/bhkim003/github_folder/ByeonghyeonKim/my_snn/net_save/save_now_net_annbase_20250108_210641_941.pth'))
    if pretrained_net != None:
        ######################## 모델이 달라서 dict로 weight만 넣고싶을 때
        # # 저장된 가중치 로드
        saved_state_dict = torch.load(pretrained_net)
        current_state_dict = net.module.state_dict()

        # 함수 호출로 가중치 매핑
        updated_state_dict = map_and_load_weights(saved_state_dict, current_state_dict)

        # 업데이트된 state_dict를 네트워크에 로드
        net.module.load_state_dict(updated_state_dict)
        ######################## 모델이 달라서 dict로 weight만 넣고싶을 때

        ############## 일반적일 때
        # net.module.load_state_dict(torch.load(pretrained_net))
        ############## 일반적일 때
    
        # pre_net = Autoencoder_conv1(input_channels=input_channels, input_length=n_sample, encoder_ch = [32, 64, 96], fc_dim = lateral_feature_num, padding = 0, stride = 2, kernel_size = 3, need_bias=need_bias, l2norm_bridge=sae_l2_norm_bridge, relu_bridge=sae_lif_bridge, activation_collector_on=False,
        #                         batch_norm_on=batch_norm_on, QCFS_neuron_on=False).to(device)
        # pre_net = torch.nn.DataParallel(net)
        # pre_net.module.load_state_dict(torch.load(pretrained_net))
        # copy_weights(pre_net.module.encoder , net.module.encoder )
        # copy_weights(pre_net.module.decoder , net.module.decoder  )

        



    wandb.watch(net, log="all", log_freq = 10)





    if SAE_net == True:
        assert 'SAE' in net.module.__class__.__name__



    net = net.to(device)
    print(f"Total number of encoder parameters: {sum(p.numel() for p in net.module.encoder.parameters())}")
    print(net)
    print('Device:',device)

    
    if optimizer == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    elif optimizer == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr = learning_rate, momentum = 0.9)
    else:
        assert False, 'optimizer를 잘못 입력했습니다.'
        
    loss_history = []


    print(f"\nStart Training, current_time = {current_time}")

    if SAE_net == True:
        assert 'SAE' in net.module.__class__.__name__
        
    k_means_acc_best = 0
    min_loss = 9999999
    min_loss_normal = 9999999
    min_loss_coarse = 9999999
    for epoch in range(max_epoch):
        print()
        l2_loss_bin= 0
        ae_train_start_time = time.time()
        running_loss = 0.0
        running_loss_normal = 0.0
        running_loss_coarse = 0.0
        iter = 0
        net.train()
        # if True or max_epoch != 1:
        wrong_element_sum = 0
        same_data_num = 0
        total_data_num = 0
        if max_epoch != 1:
            for data in train_loader:
                optimizer.zero_grad()
                total_data_num += len(data)
                data = data.to(device)
                scaling = (level_num-3)/level_num if conv1d_scaling else 1.0
                data = zero_to_one_normalize_features(data, level_num=quantize_level_num, coarse_com_config=coarse_com_config, scaling=scaling, norm01=norm01) if normalize_on else data
                spike_backup = data
                spike = data
                spike = spike.to(device) # batch, feature
                spike_for_fusion2_net = spike
                if coarse_com_mode == True and 'SAE' in net.module.__class__.__name__:
                    spike = spike.unsqueeze(2).repeat(1, 1, level_num) # spike_length == level_num # (batch, time, feature)로 변환 
                    spike = (spike > levels).to(torch.float) 

                    spike = (spike == 0).cumsum(dim=-1).eq(1).to(torch.float) if lc_adc_on == True else spike

                    # spike: batch, time, level_num
                    # levels: time, level_num
                    if Conv_net == True:
                        spike = spike.unsqueeze(-2) # batch, time, in_channel, feature or batch in_channel,feature
                        if two_channel_input == True:
                            spike_backup = spike_backup.to(device)
                            spike_backup = spike_backup.unsqueeze(2).repeat(1, 1, level_num) # spike_length == level_num # (batch, time, feature)로 변환 
                            spike_backup = (spike_backup <= levels).to(torch.float) 
                            spike_backup = (spike_backup == 1).cumsum(dim=-1).eq(1).to(torch.float) if lc_adc_on == True else spike_backup
                            spike_backup = spike_backup.unsqueeze(-2)
                            spike = torch.cat((spike, spike_backup), dim=-2)
                    assert spike.shape[0] == batch_size and spike.shape[1] == TIME
                elif 'SAE' in net.module.__class__.__name__:
                    spike = spike.unsqueeze(-1).repeat(1, 1, TIME).permute(0,2,1) # (batch, time, feature)로 변환
                    if Conv_net == True:
                        spike = spike.unsqueeze(-2) # batch, time, in_channel, feature or batch in_channel,feature
                else:
                    if Conv_net == True:
                        if coarse_com_mode == False:
                            spike = spike.unsqueeze(-2) #batch in_channel,feature
                        else:
                            spike = spike.unsqueeze(2).repeat(1, 1, level_num) # spike_length == level_num # (batch, time, feature)로 변환 
                            spike = (spike > levels).to(torch.float) 

                            spike = (spike == 0).cumsum(dim=-1).eq(1).to(torch.float) if lc_adc_on == True else spike

                    else:
                        if coarse_com_mode == False:
                            pass
                        else:
                            spike = spike.unsqueeze(2).repeat(1, 1, level_num) # spike_length == level_num # (batch, time, feature)로 변환 
                            spike = (spike > levels).to(torch.float) 

                            spike = (spike == 0).cumsum(dim=-1).eq(1).to(torch.float) if lc_adc_on == True else spike

                            # spike: batch, time, feature
                            spike = spike.reshape(spike.shape[0], -1)

                    


                    # if fusion_net == True:
                    #     spike = spikegen.rate(spike, num_steps=TIME).transpose(0, 1)

                # spike_class = net(spike) # batch, time, feature
                spike = spike.to(torch.float32)
                encoded_spike = net.module.encoder(spike)
                spike_class = net.module.decoder(encoded_spike)
                
                # # 각 레이어의 weight와 bias의 비트 수 출력
                # print("\n=== Encoder & Decoder Weight/Bias Bit Size ===")
                # for name, param in net.module.named_parameters():
                #     bit_size = param.element_size() * 8  # 바이트 크기에 8을 곱하여 비트 수 계산
                #     print(f"{name}: {bit_size} bits ({param.dtype})")


                # print('encoded_spike', spike_class)

                # for i in range (2):
                #     plot_spike(spike[i,:,0,:].cpu().numpy())
                #     # plot_spike(spike[i,:,1,:].cpu().numpy())
                #     plot_origin_spike(spike_class.squeeze()[i].cpu().detach().numpy(), min_max_y_on = True)
                # assert False
                        

                loss = 0
                loss_normal = torch.tensor(0.0)
                loss_coarse = torch.tensor(0.0)
                if coarse_com_mode == True and 'SAE' in net.module.__class__.__name__:
                    criterion = nn.MSELoss().to(device)
                    # loss1 = nn.MSELoss()(spike_class[..., 5:25], spike[..., 5:25])
                    # loss2 = nn.MSELoss()(spike_class[..., 0:5], spike[..., 0:5])
                    # loss3 = nn.MSELoss()(spike_class[..., 25:spike_length], spike[..., 25:spike_length])
                    # loss = loss1 * chan_loss_factor*2.125 + (loss2 + loss3) *(1/chan_loss_factor)/ 4 

                    # loss1 = nn.MSELoss()(spike_class[..., 5:25, :], spike[..., 5:25, :])
                    # loss2 = nn.MSELoss()(spike_class[..., 0:5, :], spike[..., 0:5, :])
                    # loss3 = nn.MSELoss()(spike_class[..., 25:spike_length, :], spike[..., 25:spike_length, :])
                    # loss = loss1 * chan_loss_factor*2.125 + (loss2 + loss3) *(1/chan_loss_factor)/ 4 
                    if fusion_net:
                        # print('1', spike.shape) # batch, time, in_channel, feature [32, 50, 1, 50]
                        
                        # ### coarse에서 ann loss 만들기 ######
                        # spike = spike.squeeze()
                        # assert two_channel_input == False
                        # zero_mask = (spike == 0)  # 0이 있는 위치
                        # first_zero_idx = torch.where(zero_mask, torch.arange(spike.shape[-1]).to(device), spike.shape[-1]-1).min(dim=-1).values
                        # spike = levels[0][first_zero_idx]
                        # # plot_origin_spike(spike[0].cpu().detach().numpy())
                        # ### coarse에서 ann loss 만들기 ######

                        ### 그냥 원래 스파이크로 ann loss 만들기 ######
                        spike = spike_for_fusion2_net
                        ### 그냥 원래 스파이크로 ann loss 만들기 ######

                        spike = spike.squeeze()
                        spike_class = spike_class.squeeze()
                        
                        ### normal loss################
                        # loss = criterion(spike_class, spike)
                        ### normal loss################
                        
                        ### chan loss################
                        loss1 = criterion(spike_class[..., 5:25], spike[..., 5:25])
                        loss2 = criterion(spike_class[..., 0:5], spike[..., 0:5])
                        loss3 = criterion(spike_class[..., 25:], spike[..., 25:])
                        loss = loss1 * chan_loss_factor*2.125 + (loss2 + loss3) *(1/chan_loss_factor)/ 4 
                        ### chan loss################


                        # #########################################
                        # # 손실 함수 정의 (예: MSELoss 사용)
                        # criterion_joke = torch.nn.MSELoss(reduction='none')  # 개별 요소별 손실을 유지

                        # # 손실 계산
                        # loss1_joke = criterion_joke(spike_class[..., 5:25], spike[..., 5:25]).mean(dim=-1)  # (batch,)
                        # loss2_joke = criterion_joke(spike_class[..., 0:5], spike[..., 0:5]).mean(dim=-1)    # (batch,)
                        # loss3_joke = criterion_joke(spike_class[..., 25:], spike[..., 25:]).mean(dim=-1)    # (batch,)

                        # # 주어진 가중치를 적용한 최종 손실
                        # loss_joke = loss1_joke * chan_loss_factor*2.125 + (loss2_joke + loss3_joke) *(1/chan_loss_factor)/ 4 

                        # # 가장 큰 손실을 갖는 샘플의 인덱스 찾기
                        # max_loss_idx_joke = torch.argmax(loss_joke)

                        # # 해당 샘플 선택
                        # selected_sample_class = spike_class[max_loss_idx_joke]
                        # selected_sample_spike = spike[max_loss_idx_joke]

                        # # 선택한 샘플의 손실 값 출력
                        # print("Index of max loss sample:", max_loss_idx_joke.item())
                        # print("Max loss value:", loss_joke[max_loss_idx_joke].item())
                        # mean_loss_joke = loss_joke.mean().item()
                        # print("Mean loss across the batch:", mean_loss_joke)

                        # # 선택한 샘플을 시각화
                        # plot_origin_spike(selected_sample_class.cpu().detach().numpy())
                        # plot_origin_spike(selected_sample_spike.cpu().detach().numpy())
                        # #########################################

                        # coarse loss ######################################################
                        loss_normal = criterion(spike_class, spike)
                        level_num_in_loss = spike_length
                        level_interval = (coarse_com_config[0] - coarse_com_config[1]) / (level_num_in_loss-1)  # max - min
                        levels = [coarse_com_config[1] + level_interval * i for i in range(level_num_in_loss)]
                        levels = torch.tensor(levels).to(torch.float).to(device)
                        # print('coarse leves', levels)
                        levels = levels.repeat(spike_length,1) 

                        spike = spike.squeeze()
                        spike_class = spike_class.squeeze()
                        # plot_origin_spike(spike_class[0].cpu().detach().numpy())
                        spike = spike.unsqueeze(2).repeat(1, 1, level_num_in_loss) 
                        spike = (spike > levels).to(torch.float) 
                        spike_class = spike_class.unsqueeze(2).repeat(1, 1, level_num_in_loss) 
                        spike_class = (spike_class > levels).to(torch.float) 
                        # spike = spike[..., 0:-3, :]
                        # spike_class = spike_class[..., 0:-3, :]
                        loss_coarse = criterion(spike_class, spike)
                        wrong_element_sum += torch.sum(torch.abs(spike - spike_class)).item() 

                        # plot_spike(spike_class[0].cpu().detach().numpy())
                        # assert False
                        # coarse loss ######################################################
                    else:
                        spike = spike.squeeze()
                        spike_class = spike_class.squeeze()
                        loss = criterion(spike_class, spike)

                    for iii in range(spike.shape[0]):
                        same_data_num = same_data_num + 1 if torch.eq(spike[iii], spike_class[iii]).all() else same_data_num
                    wrong_element_sum += torch.sum(torch.abs(spike - spike_class)).item() 

                    # spike = spike.squeeze()
                    # spike_class = spike_class.squeeze()
                    # plot_spike(spike[0].cpu().detach().numpy())
                    # plot_spike(spike_class[0].cpu().detach().numpy())
                    # print('손실 절대값 합',np.sum(np.abs(spike[0].cpu().detach().numpy() - spike_class[0].cpu().detach().numpy())))
                    # # assert False
                elif 'SAE' in net.module.__class__.__name__:
                    criterion = nn.MSELoss().to(device)
                    loss1 = criterion(spike_class[..., 5:25], spike[..., 5:25])
                    loss2 = criterion(spike_class[..., 0:5], spike[..., 0:5])
                    loss3 = criterion(spike_class[..., 25:spike_length], spike[..., 25:spike_length])
                    # loss = loss1 * 2.125 + (loss2 + loss3)/4 # chan_loss_factor = 1
                    loss = loss1 * chan_loss_factor*2.125 + (loss2 + loss3) *(1/chan_loss_factor)/ 4 
                    assert spike_length > 25, 'spike_length가 25보다 작음'
                    # wrong_element_sum += torch.sum(torch.abs(spike - spike_class)).item() 
                else:
                    criterion = nn.MSELoss().to(device)
                    loss1 = criterion(spike_class[..., 5:25], spike[..., 5:25])
                    loss2 = criterion(spike_class[..., 0:5], spike[..., 0:5])
                    loss3 = criterion(spike_class[..., 25:spike_length], spike[..., 25:spike_length])
                    loss = loss1 * chan_loss_factor*2.125 + (loss2 + loss3) *(1/chan_loss_factor)/ 4 
                    assert spike_length > 25, 'spike_length가 25보다 작음'
                    # wrong_element_sum += torch.sum(torch.abs(spike - spike_class)).item() 


                    if l2_norm_loss_weight > 0:
                        assert len(encoded_spike.shape) == 2, 'time 성분 없는 걸로'
                        l2_loss = l2_norm_loss(encoded_spike, target_norm=1.0)  # L2Norm Loss 계산, l2 1.0되게.
                        loss = loss + l2_loss*l2_norm_loss_weight
                        l2_loss_bin += l2_loss.item()

                    # coarse loss ######################################################
                    loss_normal = criterion(spike_class, spike)
                    level_num_in_loss = quantize_level_num
                    level_interval = (coarse_com_config[0] - coarse_com_config[1]) / (level_num_in_loss-1)  # max - min
                    levels = [coarse_com_config[1] + level_interval * i for i in range(level_num_in_loss)]
                    levels = torch.tensor(levels).to(torch.float).to(device)
                    levels = levels.repeat(spike_length,1) 

                    spike = spike.squeeze()
                    spike_class = spike_class.squeeze()
                    spike = spike.squeeze()
                    spike_class = spike_class.squeeze()

                    # 차원이 1이면 0차원에 크기 1인 차원 추가
                    if spike.ndimension() == 1:
                        spike = spike.unsqueeze(0)

                    if spike_class.ndimension() == 1:
                        spike_class = spike_class.unsqueeze(0)
                    # plot_origin_spike(spike_class[0].cpu().detach().numpy())
                    spike = spike.unsqueeze(2).repeat(1, 1, level_num_in_loss) 
                    spike = (spike > levels).to(torch.float) 
                    spike_class = spike_class.unsqueeze(2).repeat(1, 1, level_num_in_loss) 
                    spike_class = (spike_class > levels).to(torch.float) 
                    # spike = spike[..., 0:-3, :]
                    # spike_class = spike_class[..., 0:-3, :]
                    loss_coarse = criterion(spike_class, spike)
                    wrong_element_sum += torch.sum(torch.abs(spike - spike_class)).item() 

                    # plot_spike(spike_class[0].cpu().detach().numpy())
                    # assert False
                    # coarse loss ######################################################

                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                running_loss_normal += loss_normal.item()
                running_loss_coarse += loss_coarse.item()
                # print(f'\nepoch-{epoch}, running_loss : {running_loss:.5f}, iter percent {iter/len(train_loader)*100:.2f}%')
                iter += 1
        else:
            print('\n\n\n max_epoch 1이면 Train 안함!!!!!!!!!!!!!!!!!!!!!')
        if l2_norm_loss_weight > 0:
            print('l2_loss_bin', l2_loss_bin/len(train_loader))
        avg_loss = running_loss / len(train_loader)
        assert not np.isnan(avg_loss), f"Error: avg_loss is NaN! Running loss: {running_loss}, Length of train_loader: {len(train_loader)}"
        loss_history.append((epoch, avg_loss))
        min_loss = min(min_loss, avg_loss)
        min_loss_normal = min(min_loss_normal, running_loss_normal/len(train_loader))
        min_loss_coarse = min(min_loss_coarse, running_loss_coarse/len(train_loader))
        print(f'\nepoch-{epoch} loss : {avg_loss:.8f}, loss_normal : {running_loss_normal/len(train_loader):.8f}, loss_coarse : {running_loss_coarse/len(train_loader):.8f}, min_loss : {min_loss:.8f}, min_loss_normal : {min_loss_normal:.8f}, min_loss_coarse : {min_loss_coarse:.8f}, wrong_element_sum : {wrong_element_sum:.8f}, same_data : {100*same_data_num/(total_data_num+1e-12):.2f}%')
        print(f"ae train 실행 시간: {time.time()-ae_train_start_time:.3f}초, 전체 시작 시간 {current_time}")

        # plot_activation_distribution(net)

        if SAE_net == False and converted_net_forward == True:
            source_encoder = net.module.encoder 
            target_encoder = converted_net.module.encoder  
            copy_weights(source_encoder, target_encoder)

        k_means_acc = 0
        converted_k_means_acc = 0
        if(epoch % accuracy_check_epoch_term == 0 or epoch == 1 or epoch == max_epoch-1): 
            accuracy_check_start_time = time.time()
            print(f'\nepoch-{epoch} accuracy check')
            k_means_bin_origin_feature = []
            k_means_bin = []
            converted_k_means_bin = []
            for ds in range(dataset_num):
                # print('\n', spike_tot[ds])

                # spike = torch.tensor(torch.load(data_path[ds]))
                spike = torch.load(data_path[ds])
                label = torch.load(label_path[ds])
                unit = label[:, 0].to(torch.int)
                ch = label[:, 1].to(torch.int)
                start = label[:, 2].to(torch.int)
                end = label[:, 3].to(torch.int)
                gt = label[:, 4].to(torch.int)
                max_slope_index = label[:, 5].to(torch.int)
                ch_winner_determinant = label[:, 6]
                # unit, ch, start, end, gt, max_slope_index = map(int, label[i][:6])
                # ch_winner_determinant = label[i][6]

                label = unit

                scaling = (level_num-3)/level_num if conv1d_scaling else 1.0
                spike = zero_to_one_normalize_features(spike, level_num=quantize_level_num, coarse_com_config=coarse_com_config, scaling=scaling, norm01=norm01) if normalize_on else spike
                
                hidden_size = lateral_feature_num*TIME if 'SAE' in net.module.__class__.__name__ and SAE_hidden_nomean == True and fusion_net == False or 'SAE_FUSION5' in net.module.__class__.__name__ else lateral_feature_num
                hidden_size = lateral_feature_num if '_DR' in net.module.__class__.__name__  else hidden_size

                encoder_batch = 128
                spike_hidden = np.zeros((len(spike), hidden_size))
                converted_spike_hidden = np.zeros((len(spike), hidden_size))
                net.eval()
                with torch.no_grad():
                    now_index = 0
                    while (1):
                        now_end_index = now_index+encoder_batch if now_index+encoder_batch < len(spike) else len(spike)
                        spike_batch = spike[now_index:now_end_index] 
                        spike_torch = spike_batch
                        spike_torch = spike_torch.float()
                        spike_backup = spike_torch
                        spike_torch = spike_torch.to(device)
                        if coarse_com_mode == True and 'SAE' in net.module.__class__.__name__:
                            spike_torch = spike_torch.unsqueeze(2).repeat(1, 1, level_num) # spike_length == level_num # (batch, time, feature)로 변환 
                            spike_torch = (spike_torch > levels).to(torch.float) 
                            spike_torch = (spike_torch == 0).cumsum(dim=-1).eq(1).to(torch.float) if lc_adc_on == True else spike_torch
                            if Conv_net == True:
                                spike_torch = spike_torch.unsqueeze(-2) # batch, time, in_channel, feature or batch in_channel,feature
                                if two_channel_input == True:
                                    spike_backup = spike_backup.to(device)
                                    spike_backup = spike_backup.unsqueeze(2).repeat(1, 1, level_num) # spike_length == level_num # (batch, time, feature)로 변환 
                                    spike_backup = (spike_backup <= levels).to(torch.float) 
                                    spike_backup = (spike_backup == 1).cumsum(dim=-1).eq(1).to(torch.float) if lc_adc_on == True else spike_backup
                                    spike_backup = spike_backup.unsqueeze(-2)
                                    spike_torch = torch.cat((spike_torch, spike_backup), dim=-2)
                        elif 'SAE' in net.module.__class__.__name__:
                            spike_torch = spike_torch.unsqueeze(1).repeat(1, TIME, 1) # (batch, time, feature)로 변환
                            if Conv_net == True:
                                spike_torch = spike_torch.unsqueeze(-2) # batch, time, in_channel, feature or batch in_channel,feature
                        else:
                            # if Conv_net == True:
                            #     spike_torch = spike_torch.unsqueeze(-2) #batch in_channel,feature
                            if Conv_net == True:
                                if coarse_com_mode == False:
                                    spike_torch = spike_torch.unsqueeze(-2) #batch in_channel,feature
                                else:
                                    spike_torch = spike_torch.unsqueeze(2).repeat(1, 1, level_num) # spike_length == level_num # (batch, time, feature)로 변환 
                                    spike_torch = (spike_torch > levels).to(torch.float) 

                                    spike_torch = (spike_torch == 0).cumsum(dim=-1).eq(1).to(torch.float) if lc_adc_on == True else spike_torch

                            else:
                                if coarse_com_mode == False:
                                    pass
                                else:
                                    spike_torch = spike_torch.unsqueeze(2).repeat(1, 1, level_num) # spike_length == level_num # (batch, time, feature)로 변환 
                                    spike_torch = (spike_torch > levels).to(torch.float) 

                                    spike_torch = (spike_torch == 0).cumsum(dim=-1).eq(1).to(torch.float) if lc_adc_on == True else spike_torch

                                    # spike: batch, time, feature
                                    spike_torch = spike_torch.reshape(spike_torch.shape[0], -1)
                            if converted_net_forward == True:
                                spike_torch_spikegen = spikegen.rate(spike_torch, num_steps=TIME).transpose(0, 1)
                            # if fusion_net == True:
                            #     spike_torch = spikegen.rate(spike_torch, num_steps=TIME).transpose(0, 1)
                        ### forward #######################################################
                        inner_inf = net.module.encoder(spike_torch)
                        if SAE_net == False and converted_net_forward == True:
                            converted_inner_inf = converted_net.module.encoder(spike_torch_spikegen)
                        ### forward #######################################################
                            
                        if 'SAE' in net.module.__class__.__name__:
                            if SAE_hidden_nomean == True:
                                inner_inf = inner_inf.reshape(spike_batch.shape[0],-1)# 펼치기
                            else:
                                inner_inf = inner_inf.mean(dim=1)# Time 방향으로 평균
                            # inner_inf = F.normalize(inner_inf, p=2, dim=1)
                        spike_hidden[now_index:now_end_index] = inner_inf.cpu().detach().numpy()
                        if SAE_net == False and converted_net_forward == True:
                            converted_spike_hidden[now_index:now_end_index] = converted_inner_inf.cpu().detach().numpy()
                        now_index += encoder_batch
                        if (now_index >= len(spike)):
                            break

                origin_kmeans_accuracy = cluster_spikes_with_accuracy_torch(features= spike, true_labels=label, n_clusters=num_cluster, init_point=None)
                kmeans_accuracy = cluster_spikes_with_accuracy_torch(features= torch.tensor(spike_hidden).to(device), true_labels=label, n_clusters=num_cluster, init_point=None)
                
                
                # plot_tsne(spike_tot[ds], kmeans_accuracy, spike_hidden, label-1, n_components=2, perplexity=30, random_state=42)
                
                k_means_bin_origin_feature.append(origin_kmeans_accuracy)
                k_means_bin.append(kmeans_accuracy)
                if SAE_net == False and converted_net_forward == True:
                    converted_kmeans_accuracy = cluster_spikes_with_accuracy_torch(features= torch.tensor(converted_spike_hidden).to(device), true_labels=label-1, n_clusters=num_cluster, init_point=None)
                    converted_k_means_bin.append(converted_kmeans_accuracy)

            print(f'k_means origin feature average accuracy : {100*sum(k_means_bin_origin_feature)/(len(k_means_bin_origin_feature)+1e-12):.8f}%, total {k_means_bin_origin_feature}')

            if SAE_net == False and converted_net_forward == True:
                converted_k_means_acc = 100*sum(converted_k_means_bin)/len(converted_k_means_bin)
                print(f'converted_kmeans average accuracy : {converted_k_means_acc:.8f}%, total {converted_k_means_bin}')
            k_means_acc = 100*sum(k_means_bin)/len(k_means_bin)
            if k_means_acc > k_means_acc_best:
                # torch.save(net, f"net_save/save_now_net_{current_time}.pth")
                torch.save(net.module.state_dict(), f"net_save/save_now_net_{current_time}.pth")
                print('save model')
            
            k_means_acc_best = max(k_means_acc_best, k_means_acc)
            print(f'kmeans average accuracy best : {k_means_acc_best:.2f}%, kmeans average accuracy : {k_means_acc:.8f}%, total {k_means_bin}')
            print(f"accuracy_check 실행 시간: {time.time()-accuracy_check_start_time:.3f}초")

        wandb.log({"avg_loss": avg_loss})
        wandb.log({"k_means_acc": k_means_acc})
        wandb.log({"k_means_acc_best": k_means_acc_best})
        wandb.log({"converted_k_means_acc": converted_k_means_acc})


In [6]:

gpu = '5'
Conv_net = True # True False
SAE_net = False # True False

# hyperparameter
dataset_num = 1
spike_length = 50 # coarse_com_mode일 때는 time step이 됨.
num_cluster = 7  # 클러스터 수 설정 # 논문엔 4개라는데 여기서는 3개로 했네
training_cycle = 1400 #1400 2400 # 그 초기 몇개까지만 cluster update할지


batch_size = 32
max_epoch = 10000 # 10000 # 1
learning_rate = 0.001
normalize_on = False # True or False # 0부터1까지 normalize
need_bias = False
# first_layer_no_train = False
lif_add_at_first = False
my_seed = 42

TIME = 50 # SAE일 때만 유효. coarse_com_mode일 때는 level_num이 됨. 즉 feature 개수.
v_decay = 0.5 # -cor
v_threshold = 0.25 # -cor
v_reset = 0.0 # -cor # 10000이상 일 시 hard reset
BPTT_on = True # +cor # True False

SAE_hidden_nomean = True # True False

current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + f"_{str(int(datetime.datetime.now().microsecond / 1000)).zfill(3)}"

optimizer = 'Adam' #'Adam', 'SGD' # 둘다 준수함. loss 줄이는 거는 adam이 좋긴한데, cluster accuracy는 비슷함.

coarse_com_mode = False # True False
coarse_com_config = (0.999, -0.0) # (max, min) (0.999, -0.0) (1.0, -0.0) (2.0, -2.0) (3.0, -3.0)

sae_l2_norm_bridge = True # True False
sae_lif_bridge = False # True False

accuracy_check_epoch_term = 1

lif_add_at_last = False # True False

two_channel_input = False # True False

lateral_feature_num = 4

lc_adc_on = False # True False

converted_net_forward = False # True False

pretrained_net = None
# pretrained_net = '/home/bhkim003/github_folder/ByeonghyeonKim/my_snn/net_save/save_now_net_중요_20250110_203117_390.pth'
# pretrained_net = '/home/bhkim003/github_folder/ByeonghyeonKim/my_snn/net_save/save_now_net_중요_20250113_134126_881_이거_94나오는거.pth'
# pretrained_net = '/home/bhkim003/github_folder/ByeonghyeonKim/my_snn/net_save/save_now_net_20250205_184901_132.pth'
# pretrained_net = '/home/bhkim003/github_folder/ByeonghyeonKim/my_snn/net_save/save_now_net_20250306_134219_133.pth'
# pretrained_net = '/home/bhkim003/github_folder/ByeonghyeonKim/my_snn/net_save/save_now_net_20250304_130322_661.pth'

vth_mul_on = False # True False
batch_norm_on = False # True False

l2_norm_loss_weight = 0 #0.0001 #0.1 #  0 # 0초과면 작동

QCFS_neuron_on = False # True False

quantize_level_num = 50 # 0이면 quantize 안함. 1이상이면 그 수만큼 quantize함. # normalize_on 켜져야됨. 음수면 0~1norm안하고 quant함

fusion_net = False # True False # SAE_net False, Conv_net True로 해라. TIME 적절하게 설정해주고.
repeat_coding = False # True False #fusion_net에서 쓰이는 거임 # True면 repeat, False면 rate coding.

sae_relu_on = False # True False

conv1d_scaling = False # True False # conv1d때매 norm하고 (level_num-3)/level_num 곱해줌 # Conv_net and coarse_com_mode and normalize_on

norm01 = True # True False # normalize_on = True일 때 01norm하는지 아님 걍 quant만 하는지.

chan_loss_factor = 1

# multi_timestep_insert = (20,10) # (20,10) # None # (한번에 넣을 timestep,stride)

wandb.init(project= f'spike_sorting just run np',save_code=False)


cluster_train_system( 
    gpu = gpu,
    Conv_net = Conv_net,
    SAE_net = SAE_net,

    # hyperparameter
    dataset_num = dataset_num,
    spike_length = spike_length,
    num_cluster = num_cluster,  # 클러스터 수 설정 # 논문엔 4개라는데 여기서는 3개로 했네
    training_cycle = training_cycle, # 그 초기 몇개까지만 cluster update할지


    batch_size = batch_size,
    max_epoch = max_epoch,
    learning_rate = learning_rate,
    normalize_on = normalize_on, # True or False #이거 안 씀 # 이거 별로 안 좋은 normalize같음 # 쓸 거면 다른 거 써라.
    need_bias = need_bias,
    # first_layer_no_train = False
    lif_add_at_first = lif_add_at_first,
    my_seed = my_seed,

    TIME = TIME, # SAE일 때만 유효
    v_decay = v_decay,
    v_threshold = v_threshold,
    v_reset = v_reset, # 10000이상 일 시 hard reset
    BPTT_on = BPTT_on,

    SAE_hidden_nomean = SAE_hidden_nomean,
    
    current_time = current_time,

    optimizer = optimizer, #'Adam', 'SGD'

    coarse_com_mode = coarse_com_mode,
    coarse_com_config = coarse_com_config, # (max, min)

    
    sae_l2_norm_bridge = sae_l2_norm_bridge,
    sae_lif_bridge = sae_lif_bridge,

    accuracy_check_epoch_term = accuracy_check_epoch_term,
    
    lif_add_at_last = lif_add_at_last,

    two_channel_input = two_channel_input,

    lateral_feature_num = lateral_feature_num,

    lc_adc_on = lc_adc_on, 

    converted_net_forward = converted_net_forward,

    pretrained_net = pretrained_net,

    vth_mul_on = vth_mul_on,
    batch_norm_on = batch_norm_on,

    l2_norm_loss_weight = l2_norm_loss_weight,
    
    QCFS_neuron_on = QCFS_neuron_on, # True False

    quantize_level_num = quantize_level_num,

    fusion_net = fusion_net, # True False
    repeat_coding = repeat_coding,

    sae_relu_on = sae_relu_on,

    conv1d_scaling = conv1d_scaling,

    norm01 = norm01,

    chan_loss_factor = chan_loss_factor,
    )


{'gpu': '5', 'Conv_net': True, 'SAE_net': False, 'dataset_num': 1, 'spike_length': 50, 'num_cluster': 7, 'training_cycle': 1400, 'batch_size': 32, 'max_epoch': 10000, 'learning_rate': 0.001, 'normalize_on': False, 'need_bias': False, 'lif_add_at_first': False, 'my_seed': 42, 'TIME': 50, 'v_decay': 0.5, 'v_threshold': 0.25, 'v_reset': 0.0, 'BPTT_on': True, 'SAE_hidden_nomean': True, 'current_time': '20250320_220803_601', 'optimizer': 'Adam', 'coarse_com_mode': False, 'sae_l2_norm_bridge': True, 'sae_lif_bridge': False, 'accuracy_check_epoch_term': 1, 'lif_add_at_last': False, 'two_channel_input': False, 'lateral_feature_num': 4, 'lc_adc_on': False, 'converted_net_forward': False, 'pretrained_net': None, 'vth_mul_on': False, 'batch_norm_on': False, 'l2_norm_loss_weight': 0, 'QCFS_neuron_on': False, 'quantize_level_num': 50, 'fusion_net': False, 'repeat_coding': False, 'sae_relu_on': False, 'conv1d_scaling': False, 'norm01': True, 'chan_loss_factor': 1, 'coarse_com_config': (0.999, -0.0)}

In [ ]:
# # Sweep code


# from unittest import TextTestRunner


# unique_name_hyper = 'cluster_train_system'
# # run_name = 'spike_sorting'
# sweep_start_time =  datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + f"_{str(int(datetime.datetime.now().microsecond / 1000)).zfill(3)}"
# sweep_configuration = {
#     'method': 'grid', # 'random', 'bayes', 'grid'
#     'name': f'spike_sorting_{sweep_start_time}',
#     'metric': {'goal': 'maximize', 'name': 'k_means_acc_best'},
#     'parameters': 
#     {
#         # "gpu": {"values": ['1']},  # 이건 sweep parameter아님. hyper_iter에서 직접 설정
#         "Conv_net": {"values": [True]}, 
#         "SAE_net": {"values": [True]}, 

#         "dataset_num": {"values": [16]}, 
#         "spike_length": {"values": [50]},  
#         "num_cluster": {"values": [4]}, 
#         "training_cycle": {"values": [1400]}, # [1400, 2400]

#         "batch_size": {"values": [32]}, 
#         "max_epoch": {"values": [20]}, 
#         "learning_rate": {"values": [0.001, 0.0001]},
#         "normalize_on": {"values": [True]},
#         "need_bias": {"values": [False]}, # [True, False]

#         "lif_add_at_first": {"values": [False]}, # [True, False]
#         "my_seed": {"values": [42]}, 

#         "TIME": {"values": [50]}, #  [4,6,8,10]
#         "v_decay": {"values": [0.5]}, # [0.25,0.50,0.75]
#         "v_threshold": {"values": [0.25]}, # [0.25,0.50,0.75]
#         "v_reset": {"values": [0.0]},  # [0.0, 10000.0]
#         "BPTT_on": {"values": [True]},  # [True, False]

#         "SAE_hidden_nomean": {"values": [True]}, # [True, False]

#         # "current_time": {"values": [current_time]} #밑에서 직접설정됨.

#         "optimizer": {"values": ['Adam']}, # ['Adam', 'SGD']

#         "coarse_com_mode": {"values": [True]}, # [True, False]
#         "coarse_com_config": {"values": [(0.999, -0.0)]}, # ['Adam', 'SGD']

#         "sae_l2_norm_bridge": {"values": [True]}, # [True, False]
#         "sae_lif_bridge": {"values": [False]}, # [False, True]
        
#         "accuracy_check_epoch_term": {"values": [1]}, 

#         "lif_add_at_last": {"values": [False]},# [True, False]

#         "two_channel_input": {"values": [False]},# [True, False]

#         "lateral_feature_num": {"values": [2, 4]},# [True, False]

#         "lc_adc_on": {"values": [False]},# [True, False]
        
#         "converted_net_forward": {"values": [False]},# [True, False]

#         "pretrained_net": {"values": [None]},# [None]

#         "vth_mul_on": {"values": [False]},# [True, False]
#         "batch_norm_on": {"values": [False]},# [True, False]

#         "l2_norm_loss_weight": {"values": [0]},

#         "QCFS_neuron_on": {"values": [False]},   # [True, False]

#         "quantize_level_num": {"values": [0]}, 

#         "fusion_net": {"values": [True]}, 
#         "repeat_coding": {"values": [False]}, 

#         "sae_relu_on": {"values": [False]}, 

#         "conv1d_scaling": {"values": [False]}, 

#         "norm01": {"values": [True]}, 

#         "chan_loss_factor": {"values": [0.25,0.5,0.75,1.0,1.25,1.5,1.75,2.0,2.5,3.0,3.5]}, 

        
#      }
# }


# def hyper_iter():
#     ### my_snn control board ########################
#     wandb.init(save_code = False)
#     gpu  =  '2'
#     Conv_net  =  wandb.config.Conv_net
#     SAE_net  =  wandb.config.SAE_net

#     dataset_num  =  wandb.config.dataset_num
#     spike_length  =  wandb.config.spike_length
#     num_cluster  =  wandb.config.num_cluster
#     training_cycle  =  wandb.config.training_cycle

#     batch_size  =  wandb.config.batch_size
#     max_epoch  =  wandb.config.max_epoch
#     learning_rate  =  wandb.config.learning_rate
#     normalize_on  =  wandb.config.normalize_on
#     need_bias  =  wandb.config.need_bias

#     lif_add_at_first  =  wandb.config.lif_add_at_first
#     my_seed  =  wandb.config.my_seed


#     TIME  =  wandb.config.TIME
#     v_decay  =  wandb.config.v_decay
#     v_threshold  =  wandb.config.v_threshold
#     v_reset  =  wandb.config.v_reset
#     BPTT_on  =  wandb.config.BPTT_on

#     SAE_hidden_nomean  =  wandb.config.SAE_hidden_nomean
    
#     current_time =  datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + f"_{str(int(datetime.datetime.now().microsecond / 1000)).zfill(3)}"

#     optimizer  =  wandb.config.optimizer

#     coarse_com_mode = wandb.config.coarse_com_mode
#     coarse_com_config = wandb.config.coarse_com_config # (max, min)

#     sae_l2_norm_bridge = wandb.config.sae_l2_norm_bridge
#     sae_lif_bridge = wandb.config.sae_lif_bridge

#     accuracy_check_epoch_term = wandb.config.accuracy_check_epoch_term

#     lif_add_at_last = wandb.config.lif_add_at_last

#     two_channel_input = wandb.config.two_channel_input

#     lateral_feature_num = wandb.config.lateral_feature_num

#     lc_adc_on = wandb.config.lc_adc_on

#     converted_net_forward = wandb.config.converted_net_forward

#     pretrained_net = wandb.config.pretrained_net

#     vth_mul_on = wandb.config.vth_mul_on
#     batch_norm_on = wandb.config.batch_norm_on

#     l2_norm_loss_weight = wandb.config.l2_norm_loss_weight

#     QCFS_neuron_on = wandb.config.QCFS_neuron_on

#     quantize_level_num = wandb.config.quantize_level_num

#     fusion_net = wandb.config.fusion_net
#     repeat_coding = wandb.config.repeat_coding

#     sae_relu_on = wandb.config.sae_relu_on

#     conv1d_scaling = wandb.config.conv1d_scaling

#     norm01 = wandb.config.norm01

#     chan_loss_factor = wandb.config.chan_loss_factor

#     cluster_train_system( 
#         gpu = gpu,
#         Conv_net = Conv_net,
#         SAE_net = SAE_net,

#         # hyperparameter
#         dataset_num = dataset_num,
#         spike_length = spike_length,
#         num_cluster = num_cluster,  # 클러스터 수 설정 # 논문엔 4개라는데 여기서는 3개로 했네
#         training_cycle = training_cycle, # 그 초기 몇개까지만 cluster update할지


#         batch_size = batch_size,
#         max_epoch = max_epoch,
#         learning_rate = learning_rate,
#         normalize_on = normalize_on, # True or False #이거 안 씀 # 이거 별로 안 좋은 normalize같음 # 쓸 거면 다른 거 써라.
#         need_bias = need_bias,
#         # first_layer_no_train = False
#         lif_add_at_first = lif_add_at_first,
#         my_seed = my_seed,

#         TIME = TIME, # SAE일 때만 유효
#         v_decay = v_decay,
#         v_threshold = v_threshold,
#         v_reset = v_reset, # 10000이상 일 시 hard reset
#         BPTT_on = BPTT_on,

#         SAE_hidden_nomean = SAE_hidden_nomean,

#         current_time = current_time,

#         optimizer = optimizer, #'Adam', 'SGD'

#         coarse_com_mode = coarse_com_mode,
#         coarse_com_config = coarse_com_config, # (max, min)
        
#         sae_l2_norm_bridge = sae_l2_norm_bridge,
#         sae_lif_bridge = sae_lif_bridge,

#         accuracy_check_epoch_term = accuracy_check_epoch_term,

#         lif_add_at_last = lif_add_at_last,
        
#         two_channel_input = two_channel_input,
        
#         lateral_feature_num = lateral_feature_num,

#         lc_adc_on = lc_adc_on,

#         converted_net_forward = converted_net_forward,

#         pretrained_net = pretrained_net,

#         vth_mul_on = vth_mul_on,
#         batch_norm_on = batch_norm_on,

#         l2_norm_loss_weight = l2_norm_loss_weight,

#         QCFS_neuron_on = QCFS_neuron_on,

#         quantize_level_num = quantize_level_num,

#         fusion_net = fusion_net, 
#         repeat_coding = repeat_coding, 

#         sae_relu_on = sae_relu_on,

#         conv1d_scaling = conv1d_scaling,

#         norm01 = norm01,

#         chan_loss_factor = chan_loss_factor,
#         )
    
# # sweep_id = 'ygoj9jt4'
# sweep_id = wandb.sweep(sweep=sweep_configuration, project=f'spike_sorting {unique_name_hyper} np')
# wandb.agent(sweep_id, function=hyper_iter, count=100000, project=f'spike_sorting {unique_name_hyper}')
        


In [ ]:
wandb.finish() 

In [ ]:
# import matplotlib.pyplot as plt
# from matplotlib.ticker import MaxNLocator
# import pickle
# import json

# # current_time = '20250102_225243_972'

# with open(f"result_save/cluster_accuracy_history_{current_time}.pkl", "rb") as f:
#     data = pickle.load(f)


# # JSON으로 저장
# with open(f"result_save/cluster_accuracy_history_{current_time}.json", 'r') as f:
#     loaded_hyperparameters = json.load(f)

# loss_history = data['loss_history']
# mean_cluster_accuracy_during_training_cycle_all_dataset_history = data['mean_cluster_accuracy_during_training_cycle_all_dataset_history']
# mean_cluster_accuracy_post_training_cycle_all_dataset_history = data['mean_cluster_accuracy_post_training_cycle_all_dataset_history']
# mean_cluster_accuracy_total_all_dataset_history = data['mean_cluster_accuracy_total_all_dataset_history']
# print(data)
# max_acc = 0
# for i in mean_cluster_accuracy_post_training_cycle_all_dataset_history:
#     if i[1] > max_acc:
#         max_acc = i[1]

# # 설정 정보 제목 작성
# title = (
#     f"Dataset Num: {loaded_hyperparameters['dataset_num']}, Conv {loaded_hyperparameters['Conv_net']}, SAE {loaded_hyperparameters['SAE_net']}, Current time {loaded_hyperparameters['current_time']}, Spike Length: {loaded_hyperparameters['spike_length']}, Num Cluster: {loaded_hyperparameters['num_cluster']}, "
#     f"Training Cycle: {loaded_hyperparameters['training_cycle']}, Batch Size: {loaded_hyperparameters['batch_size']}, Max Epoch: {loaded_hyperparameters['max_epoch']}, \n"
#     f"Learning Rate: {loaded_hyperparameters['learning_rate']}, Input Normalize: {loaded_hyperparameters['normalize_on']}, Need Bias: {loaded_hyperparameters['need_bias']}, "
#     f"LIF Add at First: {loaded_hyperparameters['lif_add_at_first']}, TIME: {loaded_hyperparameters['TIME']}, Seed: {loaded_hyperparameters['my_seed']}, Best ACC: {max_acc:.2f}%"
# )

# # 데이터 리스트와 라벨 설정 (Loss 제외)
# data_list = [
#     ("Mean Cluster Accuracy (During Training Cycle)", mean_cluster_accuracy_during_training_cycle_all_dataset_history),
#     ("Mean Cluster Accuracy (Post Training Cycle)", mean_cluster_accuracy_post_training_cycle_all_dataset_history),
#     ("Mean Cluster Accuracy (Total)", mean_cluster_accuracy_total_all_dataset_history),
# ]

# # 플롯 생성
# fig, ax1 = plt.subplots(figsize=(12, 8))

# # 첫 번째 y축: Accuracy 관련 데이터
# for label, data in data_list:
#     epochs, values = zip(*data)  # epoch, value 분리
#     ax1.plot(epochs, values, label=label)

# ax1.set_xlabel("Epoch")
# ax1.set_ylabel("Clurstering Accuracy [%]", color="blue")
# ax1.tick_params(axis="y", labelcolor="blue")
# ax1.legend(loc="center right")
# ax1.grid(True)

# # x축을 정수만 표시하도록 설정
# ax1.xaxis.set_major_locator(MaxNLocator(integer=True))

# # 두 번째 y축: Loss History
# ax2 = ax1.twinx()
# epochs, values = zip(*loss_history)
# ax2.plot(epochs, values, label="AE Loss History", color="red", linestyle="--")
# ax2.set_ylabel("Loss", color="red")
# ax2.tick_params(axis="y", labelcolor="red")
# ax2.legend(loc="center left")

# # 제목 추가
# plt.title(title, fontsize=10)
# plt.tight_layout()
# plt.savefig(f'net_save/{current_time}', dpi=300, bbox_inches="tight")  # dpi=300은 고해상도로 저장, bbox_inches="tight"는 여백 최소화
# plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

def create_image_grid(image_paths, grid_size=(4, 4)):
    """
    여러 개의 이미지를 4x4 그리드로 병합하여 저장하는 함수.

    Parameters:
        image_paths (list of str): 불러올 이미지 경로 리스트 (16개 필요).
        grid_size (tuple): (rows, cols) 형태의 그리드 크기 (기본값: (4, 4)).
        save_path (str): 저장할 파일 경로.

    Returns:
        None
    """
    rows, cols = grid_size
    assert len(image_paths) == rows * cols, f"Need {rows * cols} images, but got {len(image_paths)}"

    # 이미지 불러오기
    images = [Image.open(img_path) for img_path in image_paths]

    # 모든 이미지 크기 맞추기 (첫 번째 이미지 기준)
    img_width, img_height = images[0].size
    new_image = Image.new("RGB", (cols * img_width, rows * img_height))

    # 이미지 붙이기
    for i, img in enumerate(images):
        x_offset = (i % cols) * img_width
        y_offset = (i // cols) * img_height
        new_image.paste(img, (x_offset, y_offset))

    # 시각화
    plt.figure(figsize=(32, 32))
    plt.imshow(new_image)
    plt.axis("off")
    plt.show()

# 사용 예시
    

spike_tot = ["BH_Spike_e1n005.npy", "BH_Spike_e1n010.npy", "BH_Spike_e1n015.npy", "BH_Spike_e1n020.npy",
            "BH_Spike_e2n005.npy", "BH_Spike_e2n010.npy", "BH_Spike_e2n015.npy", "BH_Spike_e2n020.npy",
            "BH_Spike_d1n005.npy", "BH_Spike_d1n010.npy", "BH_Spike_d1n015.npy", "BH_Spike_d1n020.npy",
            "BH_Spike_d2n005.npy", "BH_Spike_d2n010.npy", "BH_Spike_d2n015.npy", "BH_Spike_d2n020.npy"]
image_paths = [f"/home/bhkim003/github_folder/ByeonghyeonKim/my_snn/picture/{spike_tot[i]}.png" for i in range(len(spike_tot))]  # 저장된 16개의 이미지
create_image_grid(image_paths, grid_size=(4, 4))

